In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
path='/data/groups/CTd/'

## Read in data


In [ ]:
def process_file(f, suffix):
    tmp=pd.read_csv(path+f,low_memory=False)
    if suffix!='':
        tmp.columns=[c+suffix if c not in ['semester_clean', 'student_id'] else c for c in tmp.columns]
    return tmp

In [ ]:
fs = [
    ('procrastinator_modal_activity_range_std_D_all_phases.csv', ''),
    ('procrastinator_modal_activity_range_std_W_all_phases.csv', ''),
    ('procrastinator_modal_activity_range_std_E_all_phases.csv', ''),
    ('procrastinator_modal_activity_range_std_all_actions_all_phases.csv', ''),
    ('procrastinator_modal_activity_range_std_all_actions.csv', ''),
    ('procrastinator_modal_activity_range_std_D.csv', ''),
    ('procrastinator_modal_activity_range_std_W.csv', ''),
    ('procrastinator_modal_activity_range_std_E.csv', '')
]

In [ ]:
df_outcomes=pd.read_csv(path+'student_semester_dropped_courses_units.csv').fillna(0)
for index, (f, suff) in tqdm(enumerate(fs)):
    tmp=process_file(f, suff)
    df_outcomes=df_outcomes.merge(tmp,how='left',on=['student_id','semester_clean'])

In [ ]:
columns = ['semester_clean',
 'student_id',
 'relative_location_btw_phase1_add_drop_ddl_D_all_phases',
 'relative_location_btw_phase1_add_drop_ddl_E_all_phases',
 'relative_location_btw_phase1_add_drop_ddl_all_actions_all_phases',
 'diff_std_normalized_btw_phase1_add_drop_ddl_E_all_phases',
 'diff_std_normalized_btw_phase1_add_drop_ddl_D_all_phases',
 'diff_std_normalized_btw_phase1_add_drop_ddl_all_actions_all_phases',
 'total_dropped_units', 'total_late_dropped_units']

In [ ]:
df_outcomes_cleaned = df_outcomes[columns].copy()

In [ ]:
[c for c in df_outcomes_cleaned.columns if c in ['semester_clean', 'student_id'] or 'relative_location' in c or 'diff_std_normalized' in c]

## Control variables for regression modeling and preparing export

In [ ]:
df_outcomes_cleaned['phase1_add_drop_noE_SD'] =\
    df_outcomes_cleaned['diff_std_normalized_btw_phase1_add_drop_ddl_E_all_phases']\
    .map(lambda x: 1 if pd.isna(x) else 0)

df_outcomes_cleaned['phase1_add_drop_noD_SD'] =\
    df_outcomes_cleaned['diff_std_normalized_btw_phase1_add_drop_ddl_D_all_phases']\
    .map(lambda x: 1 if pd.isna(x) else 0)

In [ ]:
df_outcomes_cleaned['diff_std_normalized_btw_phase1_add_drop_ddl_E_all_phases'] =\
    df_outcomes_cleaned['diff_std_normalized_btw_phase1_add_drop_ddl_E_all_phases']\
    .map(lambda x: 0 if pd.isna(x) else x)

df_outcomes_cleaned['diff_std_normalized_btw_phase1_add_drop_ddl_D_all_phases'] =\
    df_outcomes_cleaned['diff_std_normalized_btw_phase1_add_drop_ddl_D_all_phases']\
    .map(lambda x: 0 if pd.isna(x) else x)

In [ ]:
# Remove NA
df_model=df_outcomes_cleaned.dropna()

## Export for R

In [ ]:
# Export for R
df_model.to_csv('df_model-procrastination-v1.csv', index=False)